# Setup

In [106]:
import os
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd

In [107]:
with open('secret/secret.txt') as f:
    api_key = f.readline().strip()

youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

# Load and Cache

In [108]:
def get_and_add_playlist_id_to_dataframe(dataframe, channel_ids):
    playlist_ids = []
    for channel_id in channel_ids:
        request = youtube.channels().list(
            part="contentDetails",
            id=channel_id
        )
        response = request.execute()
        playlist_ids.append(response['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
    dataframe['playlist_id'] = playlist_ids
    return dataframe

In [109]:
import ast

def get_video_data(video_data_path, channel_data):
    if os.path.exists(video_data_path):
        video_data = pd.read_csv(video_data_path)
    else:
        video_data = pd.DataFrame(columns=["channel_id","video_id", "video_title", "publish_date" ,"view_count", "like_count", "dislike_count", "comment_count", "default_language", "duration","has_paid_product_placement"])

        try:
            for id,channel_id in zip(channel_data["video_id"],channel_data["channel_id"]):

                video_request = youtube.videos().list(
                    part="snippet,statistics,contentDetails",
                    id=id
                )

                video_response = video_request.execute()

                for video in video_response["items"]:

                    try:
                        video_id = video["id"]
                    except KeyError:
                        video_id = float('nan')
                    try:
                        view_count = video["statistics"]["viewCount"]
                    except KeyError:
                        view_count = float('nan')
                    try:
                        like_count = video["statistics"]["likeCount"]
                    except KeyError:
                        like_count = float('nan')
                    try:
                        dislike_count = video["statistics"]["dislikeCount"]
                    except KeyError:
                        dislike_count = float('nan')
                    try:
                        comment_count = video["statistics"]["commentCount"]
                    except KeyError:
                        comment_count = float('nan')
                    try:
                        video_title = video["snippet"]["title"]
                    except KeyError:
                        video_title = float('nan')
                    try:
                        publish_date = video["snippet"]["publishedAt"]
                    except KeyError:
                        publish_date = float('nan')
                    try:
                        default_language = video["snippet"]["defaultLanguage"]
                    except KeyError:
                        default_language = float('nan')
                    try:
                        duration = video["contentDetails"]["duration"]
                    except KeyError:
                        duration = float('nan')
                    try:
                        has_paid_product_placement = video["contentDetails"]["hasPaidProductPlacement"]
                    except KeyError:
                        has_paid_product_placement = float('nan')
                    

                    row = pd.DataFrame({
                        "video_id": [video_id],
                        "channel_id": [channel_id],
                        "video_title": [video_title],
                        "view_count": [view_count],
                        "like_count": [like_count],
                        "dislike_count": [dislike_count],
                        "comment_count": [comment_count],
                        "publish_date": [publish_date],
                        "default_language": [default_language],
                        "duration": [duration],
                        "has_paid_product_placement": [has_paid_product_placement]
                    })

                    video_data = pd.concat([video_data, row], ignore_index=True)

        except Exception as e:
            print(f"Error {e}")
            video_response = {"items": []}

        if len(video_data) > 0:    
            video_data.to_csv(video_data_path, index=False)
    
    return video_data

In [110]:
channel_data_path = f"ChannelID_Data.csv"
video_data_path = f"video_data.csv"

channel_data =  pd.read_csv(channel_data_path)
video_data = pd.read_csv(video_data_path)
 

# Use Data

In [111]:
display(channel_data.head())
display(video_data.head())

,Unnamed: 0,channel_title,channel_id,description,view_count,subscriber_count,video_count,Category,playlist_id
0,0,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA
1,1,Music,UC-9-kyTW8ZkZNDHQJ6FgpwQ,Visit the YouTube Music Channel to find today’...,0,121000000,0,Music,UU-9-kyTW8ZkZNDHQJ6FgpwQ
2,2,Zee Music Company,UCFFbwnve3yF62-tVXkTyHqg,Zee Music Company is part of India's leading t...,69639694195,110000000,12150,Music,UUFFbwnve3yF62-tVXkTyHqg
3,3,BLACKPINK,UCOmHUn--16B90oW2L6FRR3A,BLACKPINK Official YouTube Channel\n블랙핑크 공식 유튜...,36756090437,94800000,603,Music,UUOmHUn--16B90oW2L6FRR3A
4,4,BANGTANTV,UCLkAepWjdylmXSltofFvsYQ,This is the official YouTube channel of BTS. 방...,23675450062,79000000,2606,Music,UULkAepWjdylmXSltofFvsYQ


,channel_id,video_id,video_title,publish_date,view_count,like_count,dislike_count,comment_count,default_language,duration,has_paid_product_placement
0,UCq-Fj5jknLsUf-MWSy4_brA,BBAyRBTfsOU,"Vaaste Song: Dhvani Bhanushali, Tanishk Bagchi...",2019-04-06T11:56:37Z,1.613914e+09,13290150.0,NaN,363862.0,NaN,PT4M27S,NaN
1,UCq-Fj5jknLsUf-MWSy4_brA,sCbbMZ-q4-I,"Lut Gaye (Full Song) Emraan Hashmi, Yukti | Ju...",2021-02-17T06:30:11Z,1.422725e+09,11124151.0,NaN,342825.0,NaN,PT4M58S,NaN
2,UCq-Fj5jknLsUf-MWSy4_brA,JFcgOboQZ08,DILBAR Lyrical | Satyameva Jayate |John Abraha...,2018-07-09T13:50:00Z,1.338395e+09,5713894.0,NaN,190363.0,NaN,PT3M19S,NaN
3,UCq-Fj5jknLsUf-MWSy4_brA,f6vY6tYvKGA,"Cham Cham Full Video | BAAGHI | Tiger Shroff, ...",2016-05-06T08:02:29Z,1.258222e+09,4196985.0,NaN,96178.0,en,PT3M52S,NaN
4,UCq-Fj5jknLsUf-MWSy4_brA,NJ1NIIdHhXs,Bum Bum Bole (Full Song) Film - Taare Zameen P...,2011-05-24T20:39:36Z,1.162780e+09,3122535.0,NaN,70381.0,en,PT4M57S,NaN


In [112]:
channel_data = channel_data.drop(columns=['Unnamed: 0'])
video_data = video_data.drop(columns=['dislike_count'])

# merge by channel_id
merged_data = pd.merge(channel_data, video_data, on="channel_id")
display(merged_data.head())

,channel_title,channel_id,description,view_count_x,subscriber_count,video_count,Category,playlist_id,video_id,video_title,publish_date,view_count_y,like_count,comment_count,default_language,duration,has_paid_product_placement
0,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,BBAyRBTfsOU,"Vaaste Song: Dhvani Bhanushali, Tanishk Bagchi...",2019-04-06T11:56:37Z,1.613914e+09,13290150.0,363862.0,NaN,PT4M27S,NaN
1,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,sCbbMZ-q4-I,"Lut Gaye (Full Song) Emraan Hashmi, Yukti | Ju...",2021-02-17T06:30:11Z,1.422725e+09,11124151.0,342825.0,NaN,PT4M58S,NaN
2,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,JFcgOboQZ08,DILBAR Lyrical | Satyameva Jayate |John Abraha...,2018-07-09T13:50:00Z,1.338395e+09,5713894.0,190363.0,NaN,PT3M19S,NaN
3,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,f6vY6tYvKGA,"Cham Cham Full Video | BAAGHI | Tiger Shroff, ...",2016-05-06T08:02:29Z,1.258222e+09,4196985.0,96178.0,en,PT3M52S,NaN
4,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,NJ1NIIdHhXs,Bum Bum Bole (Full Song) Film - Taare Zameen P...,2011-05-24T20:39:36Z,1.162780e+09,3122535.0,70381.0,en,PT4M57S,NaN


In [113]:
#most popular video in gaming
gaming_data = merged_data[merged_data["Category"] == "Gaming"]
gaming_data = gaming_data.sort_values(by="view_count_y", ascending=False)
display(gaming_data.head())

,channel_title,channel_id,description,view_count_x,subscriber_count,video_count,Category,playlist_id,video_id,video_title,publish_date,view_count_y,like_count,comment_count,default_language,duration,has_paid_product_placement
600,MrBeast Gaming,UCIPPMRA040LQr5QPyJEbmXA,MrBeast Gaming - SUBSCRIBE OR ELSE,8203618618,45300000,141,Gaming,UUIPPMRA040LQr5QPyJEbmXA,ABTdTTnnEU8,World’s Largest Explosion!,2021-04-07T18:45:24Z,220386748.0,2396426.0,83414.0,en,PT8M32S,NaN
601,MrBeast Gaming,UCIPPMRA040LQr5QPyJEbmXA,MrBeast Gaming - SUBSCRIBE OR ELSE,8203618618,45300000,141,Gaming,UUIPPMRA040LQr5QPyJEbmXA,xN8ENrfE-TY,"If You Build a House, I'll Pay For It!",2021-01-02T19:07:38Z,196920983.0,3871325.0,89653.0,en,PT10M10S,NaN
450,Mikecrack,UCqJ5zFEED1hWs0KNQCQuYdQ,"¡Hola, soy Mikecrack, el Youtuber más prro del...",19180156676,52300000,2058,Gaming,UUqJ5zFEED1hWs0KNQCQuYdQ,TaQj43Lf3tw,🎤 CHOCOLATE CON ALMENDRAS 🍫😍 CANCIÓN ORIGINAL ...,2020-09-19T18:34:42Z,177395769.0,1893994.0,99319.0,es,PT2M46S,NaN
451,Mikecrack,UCqJ5zFEED1hWs0KNQCQuYdQ,"¡Hola, soy Mikecrack, el Youtuber más prro del...",19180156676,52300000,2058,Gaming,UUqJ5zFEED1hWs0KNQCQuYdQ,SqbUUoRbH6U,¡DE COMPRAS EN EL SUPERMERCADO! 🛒😁 LAS PERRERÍ...,2020-09-19T18:00:10Z,146602542.0,2837719.0,0.0,es,PT6M52S,NaN
550,Fernanfloo,UCV4xOVpbcV8SdueDCOxLXtQ,"Algunos dicen que soy un ""CRACK"" y otros dicen...",10673600634,47900000,546,Gaming,UUV4xOVpbcV8SdueDCOxLXtQ,OTlVmfO_j0Q,EL RAP DE FERNANFLOO !!,2016-04-06T23:41:08Z,143919898.0,4886600.0,435481.0,NaN,PT3M5S,NaN
